In [1]:
import os

import numpy as np
import torch
from lib.config import cfg

import lib.dataset.dataset3D as dataUtils
from lib.vortex.vortex import Vortex
from lib.config.project_manager import ProjectManager

project = ProjectManager()
project.load('CSV Creation')

#project.create_new(
#    name = 'CSV Creation',
#    dataset2D_path = '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen',
#    dataset3D_path = '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen')

cfg = project.get_cfg()

320
Successfully loaded project CSV Creation!


In [2]:
training_set = dataUtils.VortexDataset3D(cfg, set='train')
val_set = dataUtils.VortexDataset3D(cfg, set='val')
calibPaths = [training_set.coco.dataset['calibration']['intrinsics'], 
              training_set.coco.dataset['calibration']['extrinsics']]

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [3]:
vortexNet = Vortex('inference', cfg, calibPaths, '/home/trackingsetup/Documents/Vortex/projects/handPose_test/vortex/models/Colleen_d3_v2v_ks3/Vortex-d_80.pth')

[Info] loaded weights: Vortex-d_80.pth


In [4]:
from lib.vortex.modules.efficientdet.efficientdet import EfficientDet 
efficientDet = EfficientDet('inference', cfg, '/home/trackingsetup/Documents/Vortex/projects/Test_Crop/models/efficientdet/efficientdet-d0_50.pth')

In [ ]:
import cv2
import matplotlib.pyplot as plt
img_paths = [
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_T/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_RFT/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_RBT/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_LBT/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_LFT/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_RC/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_LC/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_RFB/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_RBB/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_LBB/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_LFB/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_B/Frame_10274.jpg',
]
centerHMs = []
imgs = []
for img_path in img_paths:
    out_dict = efficientDet.predict_on_image(img_path)
    roi = out_dict['predictions'][0]['rois'][0]
    centerHM = np.array([int((roi[0]+roi[2])/2), int((roi[1]+roi[3])/2)])
    centerHMs.append(centerHM)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.
    img = (img - cfg.DATASET.MEAN) / cfg.DATASET.STD
    bbox_hw = 160
    img = img[centerHM[1]-bbox_hw:centerHM[1]+bbox_hw, centerHM[0]-bbox_hw:centerHM[0]+bbox_hw, :]
    imgs.append(img)

imgs = torch.from_numpy(np.array(imgs))
imgs = imgs.permute(0,3,1,2).view(1,12,3,320,320).cuda().float()
centerHMs = np.array(centerHMs)

center3D = torch.from_numpy(val_set.reproTool.reconstructPoint(centerHMs.transpose())).cuda()
center3D = center3D.int()
centerHMs = torch.from_numpy(centerHMs).cuda()

heatmap3D, heatmaps_padded, points3D_net = vortexNet.model(imgs, torch.unsqueeze(centerHMs,0), torch.unsqueeze(center3D, 0))
points2D = []
for point in points3D_net[0].cpu().numpy():
    points2D.append(val_set.reproTool.reprojectPoint(point))

img = cv2.cvtColor(cv2.imread(img_paths[11]), cv2.COLOR_BGR2RGB)
for point in points2D:
    cv2.circle(img, (int(point[11][0]), int(point[11][1])), 2, (255,0,0), thickness=5)
plt.imshow(img)
plt.imsave('test.png',img)
#for img_path in img_paths:
#    out_dict = efficientDet.predict_on_image(img_path)
#    print (out_dice)
#    efficientDet.visualize_prediction(out_dict)

In [ ]:
import cv2
import csv
import matplotlib.pyplot as plt
import itertools


video_paths = [
    '/media/trackingsetup/Elements/Recordings/Recording_Colleen_03032021/Camera_T.avi',
    '/media/trackingsetup/Elements/Recordings/Recording_Colleen_03032021/Camera_RFT.avi',
    '/media/trackingsetup/Elements/Recordings/Recording_Colleen_03032021/Camera_RBT.avi',
    '/media/trackingsetup/Elements/Recordings/Recording_Colleen_03032021/Camera_LBT.avi',
    '/media/trackingsetup/Elements/Recordings/Recording_Colleen_03032021/Camera_LFT.avi',
    '/media/trackingsetup/Elements/Recordings/Recording_Colleen_03032021/Camera_RC.avi',
    '/media/trackingsetup/Elements/Recordings/Recording_Colleen_03032021/Camera_LC.avi',
    '/media/trackingsetup/Elements/Recordings/Recording_Colleen_03032021/Camera_RFB.avi',
    '/media/trackingsetup/Elements/Recordings/Recording_Colleen_03032021/Camera_RBB.avi',
    '/media/trackingsetup/Elements/Recordings/Recording_Colleen_03032021/Camera_LBB.avi',
    '/media/trackingsetup/Elements/Recordings/Recording_Colleen_03032021/Camera_LFB.avi',
    '/media/trackingsetup/Elements/Recordings/Recording_Colleen_03032021/Camera_B.avi', 
]


caps = []
outs = []
for path in video_paths:
    caps.append(cv2.VideoCapture(path))
    #outs.append(cv2.VideoWriter(os.path.join('/home/trackingsetup/Documents/Vortex/Videos_Test', path.split('/')[-1]), cv2.VideoWriter_fourcc('M','J', 'P', 'G'), 100, (1280,1024)))

for cap in caps:
        cap.set(1,0)
        
counter = 0
    
with open('3D Data/combined.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                                quotechar='"', quoting=csv.QUOTE_MINIMAL)
    header = ['Frame Idx']
    joints = ["Pinky_T","Pinky_D","Pinky_M","Pinky_P","Ring_T","Ring_D","Ring_M","Ring_P","Middle_T","Middle_D","Middle_M","Middle_P","Index_T","Index_D","Index_M","Index_P","Thumb_T","Thumb_D","Thumb_M","Thumb_P","Palm", "Wrist_U","Wrist_R"]
    joints = joints
    joints = list(itertools.chain.from_iterable(itertools.repeat(x, 3) for x in joints))
    header = header + joints
    header2 = [""]
    header2 = header2 + ['x','y', 'z']*23
    print (header2)
    writer.writerow(header)
    writer.writerow(header2)

    ret = True
    while ret:
        if counter % 20 == 0:
            print (counter)
        counter += 1
        imgs = []
        imgs_orig = []
        centerHMs = []
        for cap in caps:
            ret, img = cap.read()
            if not ret:
                break
            imgs_orig.append(img)
            out_dict = efficientDet.predict_on_image(None, [img])
            #efficientDet.visualize_prediction(out_dict)
            try:
                roi = out_dict['predictions'][0]['rois'][0]
                prev_roi = roi
            except: 
                roi = prev_roi
            centerHM = np.array([max(min(int((roi[0]+roi[2])/2),1280-160), 160), 
                                 max(min(int((roi[1]+roi[3])/2), 1024-160),160)])
            centerHMs.append(centerHM)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img.astype(np.float32) / 255.
            img = (img - cfg.DATASET.MEAN) / cfg.DATASET.STD
            bbox_hw = 160
            img = img[centerHM[1]-bbox_hw:centerHM[1]+bbox_hw, centerHM[0]-bbox_hw:centerHM[0]+bbox_hw, :]
            imgs.append(img)

        if not ret:
            break
        imgs = torch.from_numpy(np.array(imgs))
        imgs = imgs.permute(0,3,1,2).view(1,12,3,320,320).cuda().float()
        centerHMs = np.array(centerHMs)

        center3D = torch.from_numpy(val_set.reproTool.reconstructPoint(centerHMs.transpose())).cuda()
        center3D = center3D.int()
        centerHMs = torch.from_numpy(centerHMs).cuda()

        heatmap3D, heatmaps_padded, points3D_net = vortexNet.model(imgs, torch.unsqueeze(centerHMs,0), torch.unsqueeze(center3D, 0))
        row = [counter]
        for point in points3D_net.squeeze():
            row = row + point.tolist()
        writer.writerow(row)
        #points2D = []
        #for point in points3D_net[0].cpu().numpy():
        #    points2D.append(val_set.reproTool.reprojectPoint(point))

        #colors = [(255,0,0), (255,0,0),(255,0,0),(255,0,0),(0,255,0),(0,255,0),(0,255,0),(0,255,0),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(255,255,0),(255,255,0),(255,255,0), (255,255,0),
        #          (0,255,255),(0,255,255),(0,255,255),(0,255,255), (255,0,255),(100,0,100),(100,0,100)]

        #for i,out in enumerate(outs):
        #    img = imgs_orig[i]
        #    for j,point in enumerate(points2D):
        #        cv2.circle(img, (int(point[i][0]), int(point[i][1])), 2, colors[j], thickness=5)
        #    out.write(img)

    

['', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z', 'x', 'y', 'z']
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2

28000
28020
28040
28060
28080
28100
28120
28140
28160
28180
28200
28220
28240
28260
28280
28300
28320
28340
28360
28380
28400
28420
28440
28460
28480
28500
28520
28540
28560
28580
28600
28620
28640
28660
28680
28700
28720
28740
28760
28780
28800
28820
28840
28860
28880
28900
28920
28940
28960
28980
29000
29020
29040
29060
29080
29100
29120
29140
29160
29180
29200
29220
29240
29260
29280
29300
29320
29340
29360
29380
29400
29420
29440
29460
29480
29500
29520
29540
29560
29580
29600
29620
29640
29660
29680
29700
29720
29740
29760
29780
29800
29820
29840
29860
29880
29900
29920
29940
29960
29980
30000
30020
30040
30060
30080
30100
30120
30140
30160
30180
30200
30220
30240
30260
30280
30300
30320
30340
30360
30380
30400
30420
30440
30460
30480
30500
30520
30540
30560
30580
30600
30620
30640
30660
30680
30700
30720
30740
30760
30780
30800
30820
30840
30860
30880
30900
30920
30940
30960
30980
31000
31020
31040
31060
31080
31100
31120
31140
31160
31180
31200
31220
31240
31260
31280
31300
3132

55320
55340
55360
55380
55400
55420
55440
55460
55480
55500
55520
55540
55560
55580
55600
55620
55640
55660
55680
55700
55720
55740
55760
55780
55800
55820
55840
55860
55880
55900
55920
55940
55960
55980
56000
56020
56040
56060
56080
56100
56120
56140
56160
56180
56200
56220
56240
56260
56280
56300
56320
56340
56360
56380
56400
56420
56440
56460
56480
56500
56520
56540
56560
56580
56600
56620
56640
56660
56680
56700
56720
56740
56760
56780
56800
56820
56840
56860
56880
56900
56920
56940
56960
56980
57000
57020
57040
57060
57080
57100
57120
57140
57160
57180
57200
57220
57240
57260
57280
57300
57320
57340
57360
57380
57400
57420
57440
57460
57480
57500
57520
57540
57560
57580
57600
57620
57640
57660
57680
57700
57720
57740
57760
57780
57800
57820
57840
57860
57880
57900
57920
57940
57960
57980
58000
58020
58040
58060
58080
58100
58120
58140
58160
58180
58200
58220
58240
58260
58280
58300
58320
58340
58360
58380
58400
58420
58440
58460
58480
58500
58520
58540
58560
58580
58600
58620
5864

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm

figure = plt.figure()
axes = figure.gca(projection='3d')

c = ['r', 'r','r','r','b','b','b','b','g','g','g','g', 'orange', 'orange','orange','orange', 'y','y','y','y','purple', 'purple','purple']
line_idxs = [[0,1], [1,2], [2,3], [4,5], [5,6], [6,7], [8,9], [9,10], [10,11], [12,13], [13,14], [14,15], [16,17], [17,18], [18,19], [3,7], [7,11], [11,15], [3,21], [7,21],[11,22], [15,22],[21,22], [18,15], [19,22]]
points3D = points3D_net[0].cpu().numpy()
center3D = center3D.cpu()
for i, point in enumerate(points3D):
    #print ("Classic:", i, point)
    if i != 20:
        axes.scatter(point[0], point[1], point[2], color = c[i])
for line in line_idxs:
    axes.plot([points3D[line[0]][0], points3D[line[1]][0]], [points3D[line[0]][1], points3D[line[1]][1]], [points3D[line[0]][2], points3D[line[1]][2]], c = 'gray')

axes.set_xlim3d(center3D[0]-100, center3D[0]+100)
axes.set_ylim3d(center3D[1]-100, center3D[1]+100)
axes.set_zlim3d(center3D[2]-100, center3D[2]+100)
plt.subplots_adjust(left=0., right=1., top=1., bottom=0.)
#plt.gca().invert_zaxis()
plt.savefig('test.png')
plt.show()